# Cleaning data
Data cleaning is the process of fixing or removing incorrect, corrupted, duplicate, or incomplete data within a dataset. Messy data leads to unreliable outcomes. Cleaning data is an essential part of data analysis

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('AB_NYC_2019.csv')

# Initial exploration
print(f"Dataset shape: {df.shape}")
print("\nData types and missing values:")
print(df.info())
print("\nSummary statistics:")
print(df.describe(include='all'))

Dataset shape: (48895, 16)

Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48

In [2]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])

# Handle missing values in key columns
# For reviews_per_month, fill 0 for listings with no reviews
df['reviews_per_month'].fillna(0, inplace=True)

# For last_review, leave as NaN (these listings have no reviews)
# For name and host_name, very few missing - we can drop these rows
df.dropna(subset=['name', 'host_name'], inplace=True)

Missing values per column:
name                    16
host_name               21
last_review          10052
reviews_per_month    10052
dtype: int64


C:\Users\KIIT\AppData\Local\Temp\ipykernel_12352\2186338361.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews_per_month'].fillna(0, inplace=True)


In [3]:
# Check for duplicate listings
print(f"Potential duplicate listings: {df.duplicated().sum()}")

# Check for duplicate IDs (should be unique)
print(f"Duplicate IDs: {df['id'].duplicated().sum()}")

# No duplicates found in this dataset

Potential duplicate listings: 0
Duplicate IDs: 0


In [4]:
# Standardize text columns
df['name'] = df['name'].str.strip()
df['host_name'] = df['host_name'].str.strip()
df['neighbourhood_group'] = df['neighbourhood_group'].str.strip()
df['neighbourhood'] = df['neighbourhood'].str.strip()
df['room_type'] = df['room_type'].str.strip()

# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

# Standardize price (ensure no negative values)
df = df[df['price'] >= 0]

In [5]:
# Standardize text columns
df['name'] = df['name'].str.strip()
df['host_name'] = df['host_name'].str.strip()
df['neighbourhood_group'] = df['neighbourhood_group'].str.strip()
df['neighbourhood'] = df['neighbourhood'].str.strip()
df['room_type'] = df['room_type'].str.strip()

# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

# Standardize price (ensure no negative values)
df = df[df['price'] >= 0]

In [6]:
# Validate latitude and longitude are within NYC bounds
nyc_lat_range = (40.5, 40.9)
nyc_lon_range = (-74.3, -73.7)

df = df[
    (df['latitude'].between(*nyc_lat_range)) & 
    (df['longitude'].between(*nyc_lon_range))
]

# Validate room_type has only expected values
valid_room_types = ['Private room', 'Entire home/apt', 'Shared room']
df = df[df['room_type'].isin(valid_room_types)]

In [7]:
# Create new features that might be useful for analysis
df['has_availability'] = df['availability_365'] > 0
df['years_hosting'] = 2019 - pd.to_datetime(df['last_review']).dt.year

In [8]:
print("\nFinal data quality check:")
print(f"Rows: {len(df)}")
print(f"Columns: {len(df.columns)}")
print("\nMissing values:")
print(df.isnull().sum())
print("\nData types:")
print(df.dtypes)


Final data quality check:
Rows: 48840
Columns: 18

Missing values:
id                                    0
name                                  0
host_id                               0
host_name                             0
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10030
reviews_per_month                     0
calculated_host_listings_count        0
availability_365                      0
has_availability                      0
years_hosting                     10030
dtype: int64

Data types:
id                                         int64
name                                      object
host_id                                    int64
host_name                              

In [9]:
# Save cleaned data to new CSV
df.to_csv('AB_NYC_2019_cleaned.csv', index=False)
print("Cleaned data saved successfully!")

Cleaned data saved successfully!
